### Nutzung der Helper-Klasse

In [1]:
from db_communication import db_communication
import pandas as pd
import numpy as np

In [2]:
config = {
    'user': 'root',
    'password': 'FDS-apm1',
    'host': 'min-ifm-xdm.ad.fh-bielefeld.de',
    'port': '3306',
    'database': 'NeueDB' 
}
my_db = db_communication(config)

Successfully connected.


In [3]:
tables = my_db.read_tables()
tables = [tables[i][0] for i in range(len(tables))]
tables

['Auftrag',
 'Einzelteil',
 'Konfiguration',
 'Kunde',
 'Merkmalcluster',
 'Merkmale',
 'Stuecklistenteil']

### Funktionen zur Ermittlung zufälliger Attribute

In [4]:
import random
def randomElementInList(values):
    """
    Auswahl eines zufälligen Elements aus dem Wertebereich values
    Bsp.: ['Basic','Holland','BMX','Klassisch','Einrad'] ->  return = 'BMX'
    """
    return random.choice(values)

In [5]:
randomElementInList(['Basic','Holland','BMX','Klassisch','Einrad'])

'BMX'

In [6]:
def randomNumber(start,stop=None,step=1):
    """
    Generierung einer zufälligen Zahl nach random.randrange():
    Choose a random item from range(start, stop[, step])
    """
    if start != stop:
        return random.randrange(start=start,stop=stop,step=step)
    else:
        return start

In [7]:
randomNumber(1,20,1)

7

In [8]:
import time 
import datetime
def randomDate(start, end=datetime.datetime.now().strftime("%d.%m.%Y"), format="%d.%m.%Y", prop=random.random()):
    """Get a time at a proportion of a range of two formatted times.

    start and end should be strings specifying times formated in the
    given format (strftime-style), giving an interval [start, end].
    prop specifies how a proportion of the interval to be taken after
    start.  The returned time will be in the specified format.
    """

    stime = time.mktime(time.strptime(start, format))
    etime = time.mktime(time.strptime(end, format))

    ptime = stime + prop * (etime - stime)

    return time.strftime(format, time.localtime(ptime))

In [9]:
randomDate("12.10.2020","12.10.2022","%d.%m.%Y",random.random())

'12.10.2021'

In [10]:
def randomName():
    preNames = ["Tana","Francoise","Emmie","Stuart","Cedrick","Tiffani","Kym","Woodrow","Jolene","Autumn","Moriah","Claud","Antoinette",
                "Gisele","Chrissy","Ellena","Hilton","Susanne","Lindy","Nakesha","Rebecca","Lavenia","Donya","Claudia","Dreama","Alla",
                "Freeman","Weston","Jacki","Helene"]
    lastName = ["Allinder","Longley","Dobyns","Coldiron","Justin","Munroe","Boll","Clingerman","Pries","Santangelo","Miller","Bohman","Meachum",
                "Kamer","Waits","Polin","Suitt","Saulter","Pelt","Dapolito","Guild","Beeks","Hollingshead","Schissler","Cardamone","Sypher",
                "Fannin","Belizaire","Burleigh","Eslinger"]
    return randomElementInList(preNames), randomElementInList(lastName)

In [11]:
randomName()

('Claudia', 'Pries')

In [12]:
def randomMail(preName,lastName):
    providers = ['gmail.com','web.de','telekom.de','apple.com','fh-bielefeld.de']
    mail = "{0}.{1}@{2}".format(preName.lower(),lastName.lower(),randomElementInList(providers).lower())
    return mail

In [13]:
randomMail(*randomName())

'ellena.dapolito@fh-bielefeld.de'

In [14]:
def randomBike(featureTableName='Merkmale',featureClusterTableName='Merkmalcluster'):
    """
    Generierung eines zufälligen Fahrrad auf Basis der Inhalte aus den Tabellen:
    - featureCluster (Bestandteile eines Fahrrads: Rahmen, Gabel, etc.)
    - feature        (Wie diese Bestandteile umgesetzt werden: Trofeo 5, Carbon Öldruckstoßdämpfer, etc.)
    Und Bestimmung des Gesamtpreises anhand gewählter Merkmale
    """
    # get information about the parts to build a bike from DB
    df_features = my_db.get_table(featureTableName)
    df_featureCluster = my_db.get_table(featureClusterTableName)
    df_featureCluster.set_index('ClusterNr',inplace=True)
    
    # merge both dfs
    arrays = [df_features['Merkmalcluster'].replace(df_featureCluster.to_dict()['Merkmalsbezeichnung']).values, 
              df_features['MerkmalNr'].values]
    tuples = list(zip(*arrays))
    index = pd.MultiIndex.from_tuples(tuples, names=['Merkmalcluster', 'MerkmalNr'])
    df = df_features.set_index(index)
    
    # build bike
    bike = []
    bikePrice = 0
    for cluster in df['Merkmalcluster'].unique():
        # get set of Merkmale
        setMerkmale = df.query("Merkmalcluster == " + str(cluster))['MerkmalNr'].values    
        # ignoe 'no ...' part (marked with MerkmalNr : 'xxx99999')
        unvalidClusters = [element for element in setMerkmale.astype(str) if '99999' in element]
        setMerkmale = setMerkmale[setMerkmale.astype(str) != unvalidClusters]
        # select random bike part
        bikePart = randomElementInList(setMerkmale)
        # get price of this part
        pricePart = df.swaplevel().loc[int(bikePart)]['Preis'].astype(float).iloc[0]
        # sum it up to get the total bike price
        bikePrice += pricePart
        bike.append(bikePart)
    return bike, bikePrice

In [15]:
randomBike()

([100003,
  200001,
  300001,
  400001,
  500003,
  600001,
  700001,
  800002,
  900003,
  1000004],
 992.96)

## Fill data

### Kundendaten
Im ersten Schritt ist es erforderliche einen Kundendatenstamm anzulegen, welcher unabhängig befüllt werden kann und auf den weitere Bestellungen bzw. Konfigurationen zugreifen können.

In [16]:
def fillCustomerTable(n,columnNamesDB=['Name','Vorname','Mail'],includeID=True):
    """
    Hinzufügen von zufällig generierten Einträgen in der DB-Tabelle 'Kunde'
    
    n - Anzahl der Einträge die hinzugefügt werden sollen
    columnNamesDB - Spaltennamen der Kunden-Tabelle in der DB (mit Ausnahme der KundenNr; Reihenfolge beachten!) 
    includeID - Ob KundenNr seperat mit angelegt werden soll
    
    Es werden die Inhalte nach den defaults in den Parametern erzeugt!
    Das Ändern der Reihenfolge der Spalten und die Hinzunahme weiterer Spalten muss im Quellcode angepasst werden!
    """
    # Kundentabelle füllen
    table_name = 'Kunde'
    customerId = 'KundenNr'
    if includeID:
        customer_keys = np.concatenate(([customerId],columnNamesDB)) 
    else:
        customer_keys = columnNamesDB
    for i in range(n):
        customer_name = randomName()
        customer_values = i+1, customer_name[0], customer_name[1], randomMail(*customer_name)
        if includeID:
            customer_dict = dict(zip(customer_keys,customer_values))
        else:
            customer_dict = dict(zip(customer_keys,customer_values[1:]))
        my_db.insert_data(table_name,customer_dict)

In [17]:
fillCustomerTable(1,includeID=False)

INSERT INTO `Kunde` (`Name`, `Vorname`, `Mail`) VALUES ('Moriah', 'Hollingshead', 'moriah.hollingshead@web.de');


### Konfiguraton
Im zweiten Schritt werden zufällige Konfigurationen von Fahrrädern erstellt, die einer Auftragsnummer hinzugefügt werden und dessen Gesamtpreis berechnet wird.

In [34]:
def generateConfigurationData(n,columnNamesDB=['AuftragNr','MerkmalNr'],table_name='Konfiguration',
                              configId_name = 'KonfigNr',includeID=True):
    """
    Aufgrund der Abhängigkeiten der Tabellen 'Auftrag' und 'Konfiguration' mit den Spalten 'Gesamtpreis' <-> 'AuftragNr', 
    wird in dieser Funktion der Output für die Konfiguration inkl. Informationen für die  für ein späteres Einfügen in die DB erzeugt
    
    """
    # create keys
    if includeID:
        config_keys = np.concatenate(([configId_name],columnNamesDB)) 
    else:
        config_keys = columnNamesDB

    # get old entries in order to appending new ones
    df_before = my_db.get_table(table_name)
    start = df_before.iloc[:,1].max() + 1 #max orderId + 1
    n += start

    order = []
    config_dict = []
    # iterate through all orders that should be created
    for orderId in range(start,n):
        # build random bike
        bike,bikePrice = randomBike()
        # keep orderId and bikePrice for later order fillings
        order.append(dict(zip(('orderId','bikePrice'),(orderId,bikePrice))))
        # iterate through each bike part
        for partId, featureId in enumerate(bike):
            # generate values
            config_values = orderId*100+partId, orderId, featureId
            # keep the config_dict for later inserts 
            if includeID:
                config_dict.append(dict(zip(config_keys,config_values)))
            else:
                config_dict.append(dict(zip(config_keys,config_values[1:])))
                
    df_order = pd.DataFrame(order)
    return config_dict, df_order 

In [36]:
generateConfigurationData(1)

[{'KonfigNr': 400, 'AuftragNr': 4, 'MerkmalNr': 100003}, {'KonfigNr': 401, 'AuftragNr': 4, 'MerkmalNr': 200002}, {'KonfigNr': 402, 'AuftragNr': 4, 'MerkmalNr': 300004}, {'KonfigNr': 403, 'AuftragNr': 4, 'MerkmalNr': 400002}, {'KonfigNr': 404, 'AuftragNr': 4, 'MerkmalNr': 500005}, {'KonfigNr': 405, 'AuftragNr': 4, 'MerkmalNr': 600002}, {'KonfigNr': 406, 'AuftragNr': 4, 'MerkmalNr': 700003}, {'KonfigNr': 407, 'AuftragNr': 4, 'MerkmalNr': 800002}, {'KonfigNr': 408, 'AuftragNr': 4, 'MerkmalNr': 900003}, {'KonfigNr': 409, 'AuftragNr': 4, 'MerkmalNr': 1000004}]


,orderId,bikePrice
0,4,1065.47


In [28]:
n = 1
columnNamesDB=['AuftragNr','MerkmalNr']
table_name = 'Konfiguration'
configId_name = 'KonfigNr'
includeID = True

# create keys
if includeID:
    order_keys = np.concatenate(([configId_name],columnNamesDB)) 
else:
    order_keys = columnNamesDB

# get old entries in order to appending new ones
df_before = my_db.get_table(table_name)
start = df_before.iloc[:,1].max() + 1 #max orderId + 1
n += start

order = []
# iterate through all orders that should be created
for orderId in range(start,n):
    print(orderId)
    # build random bike
    bike,bikePrice = randomBike()
    # keep orderId and bikePrice for later order fillings
    order.append(dict(zip(('orderId','bikePrice'),(orderId,bikePrice))))
    # iterate through each bike part
    for partId, featureId in enumerate(bike):
        # generate values
        order_values = orderId*100+partId, orderId, featureId
        if includeID:
            order_dict = dict(zip(order_keys,order_values))
        else:
            order_dict = dict(zip(order_keys,order_values[1:]))
        # insert each bike part as a new row
        print(order_dict)
        my_db.insert_data(table_name,order_dict)
        
df_order = pd.DataFrame(order)
#return df_order
        

2
{'KonfigNr': 200, 'AuftragNr': 2, 'MerkmalNr': 100002}
INSERT INTO `Konfiguration` (`KonfigNr`, `AuftragNr`, `MerkmalNr`) VALUES ('200', '2', '100002');
{'KonfigNr': 201, 'AuftragNr': 2, 'MerkmalNr': 200003}
INSERT INTO `Konfiguration` (`KonfigNr`, `AuftragNr`, `MerkmalNr`) VALUES ('201', '2', '200003');
{'KonfigNr': 202, 'AuftragNr': 2, 'MerkmalNr': 300001}
INSERT INTO `Konfiguration` (`KonfigNr`, `AuftragNr`, `MerkmalNr`) VALUES ('202', '2', '300001');
{'KonfigNr': 203, 'AuftragNr': 2, 'MerkmalNr': 400003}
INSERT INTO `Konfiguration` (`KonfigNr`, `AuftragNr`, `MerkmalNr`) VALUES ('203', '2', '400003');
{'KonfigNr': 204, 'AuftragNr': 2, 'MerkmalNr': 500002}
INSERT INTO `Konfiguration` (`KonfigNr`, `AuftragNr`, `MerkmalNr`) VALUES ('204', '2', '500002');
{'KonfigNr': 205, 'AuftragNr': 2, 'MerkmalNr': 600002}
INSERT INTO `Konfiguration` (`KonfigNr`, `AuftragNr`, `MerkmalNr`) VALUES ('205', '2', '600002');
{'KonfigNr': 206, 'AuftragNr': 2, 'MerkmalNr': 700001}
INSERT INTO `Konfiguratio

In [25]:
def fillOrderTable(df_order,columnNamesDB=['Bestelldatum','Lieferdatum','KundenNr','Gesamtpreis'],
                   includeID=True, customerTableName='Kunde',customerId='KundenNr',startDate='2020-01-01'):
    """
    Hinzufügen von zufällig generierten Einträgen in der DB-Tabelle 'Aufträge'
    
    n - Anzahl der Einträge die hinzugefügt werden sollen
    df_order - DataFrames welches von dem Konfigurator zur Anlegung eines Auftrags und zur Bestimmung des Gesamtpreises verwendet wird
    columnNamesDB - Spaltennamen der Auftrag-Tabelle in der DB (mit Ausnahme der AuftrNr; Reihenfolge beachten!) 
    includeID - Ob AuftrNr seperat mit angelegt werden soll
    customerTableName - Name der DB-Tabelle 'Kunde'
    customerId - Name der Identifikationsnummer in customerTable
    startDate - Startdatum zur zufälligen Erzeugung
    
    Es werden die Inhalte nach den defaults in den Parametern erzeugt!
    Das Ändern der Reihenfolge der Spalten und die Hinzunahme weiterer Spalten muss im Quellcode angepasst werden!
    """
    table_name = 'Auftrag'
    orderId_name = 'AuftrNr'
    
    # Read relevant range of values for existing customers
    df_customers = my_db.get_table(customerTableName)
    
    # create keys
    if includeID:
        order_keys = np.concatenate(([orderId_name],columnNamesDB)) 
    else:
        order_keys = columnNamesDB
        
    # create values and fill
    for _,order in df_order.iterrows():
        orderId = order[0].astype(int)
        orderPrice = order[1].astype(float)
        
        rand_orderDate = randomDate(start=startDate,
                                    end=datetime.datetime.now().strftime("%Y-%m-%d"),
                                    format="%Y-%m-%d",
                                    prop=random.random())
        rand_delivDate = datetime.datetime.strftime(datetime.datetime.strptime(rand_orderDate,"%Y-%m-%d") + datetime.timedelta(days=14),"%Y-%m-%d")
        rand_customerId = randomNumber(start=df_customers[customerId].astype(int).min(),
                                        stop=df_customers[customerId].astype(int).max())
        order_values = orderId, rand_orderDate, rand_delivDate, rand_customerId, orderPrice
        if includeID:
            order_dict = dict(zip(order_keys,order_values))
        else:
            order_dict = dict(zip(order_keys,order_values[1:]))
        my_db.insert_data(table_name,order_dict)

In [ ]:
def fill

In [26]:
fillOrderTable(1,df_order,includeID=False)

INSERT INTO `Auftrag` (`Bestelldatum`, `Lieferdatum`, `KundenNr`, `Gesamtpreis`) VALUES ('2020-05-30', '2020-06-13', '2', '1037.97');
INSERT INTO `Auftrag` (`Bestelldatum`, `Lieferdatum`, `KundenNr`, `Gesamtpreis`) VALUES ('2020-03-15', '2020-03-29', '3', '1099.95');


In [27]:
df_order

,orderId,bikePrice
0,2,1037.97
1,3,1099.95


In [ ]:
for _,order in df_order.iterrows():
    print(order[0].astype(int))

### Aufträge

In [ ]:
def fillOrderTable(n,columnNamesDB=['Datum','KundenNr'],includeID=True,
                   customerTableName='Kunde',customerId='KundenNr',startDate='2020-01-01'):
    """
    Hinzufügen von zufällig generierten Einträgen in der DB-Tabelle 'Aufträge'
    
    n - Anzahl der Einträge die hinzugefügt werden sollen
    columnNamesDB - Spaltennamen der Auftrag-Tabelle in der DB (mit Ausnahme der AuftrNr; Reihenfolge beachten!) 
    includeID - Ob AuftrNr seperat mit angelegt werden soll
    customerTableName - Name der DB-Tabelle 'Kunde'
    customerId - Name der Identifikationsnummer in customerTable
    startDate - Startdatum zur zufälligen Erzeugung
    
    Es werden die Inhalte nach den defaults in den Parametern erzeugt!
    Das Ändern der Reihenfolge der Spalten und die Hinzunahme weiterer Spalten muss im Quellcode angepasst werden!
    """
    table_name = 'Auftrag'
    orderId = 'AuftrNr'
    
    # Lesen der Wertebereiche aus relevanten Tabellen
    df_customers = my_db.get_table(customerTableName)
    
    # Auftragstabelle füllen
    if includeID:
        order_keys = np.concatenate(([customerId],columnNamesDB)) 
    else:
        order_keys = columnNamesDB
    for i in range(n):
        rand_orderDate = randomDate(start=startDate,
                                    end=datetime.datetime.now().strftime("%Y-%m-%d"),
                                    format="%Y-%m-%d",
                                    prop=random.random())
        rand_customerId = randomNumber(start=df_customers[customerId].astype(int).min(),
                                        stop=df_customers[customerId].astype(int).max())
        order_values = i+1, rand_orderDate, rand_customerId
        if includeID:
            order_dict = dict(zip(order_keys,order_values))
        else:
            order_dict = dict(zip(order_keys,order_values[1:]))
        my_db.insert_data(table_name,order_dict)

In [ ]:
fillOrderTable(1)

### Konfigurator TEST

In [ ]:
# TODO: replace this by read tabels from DB
df_merkmalcluster = pd.DataFrame(merkmalcluster)
df_merkmalcluster.set_index('ClusterNr',inplace=True)
df_merkmale = pd.DataFrame(merkmale)

In [ ]:
#merge dfs
arrays = [df_merkmale['Merkmalcluster'].replace(df_merkmalcluster.to_dict()['Merkmalsbezeichnung']).values, 
          df_merkmale['MerkmalNr'].values]
tuples = list(zip(*arrays))
index = pd.MultiIndex.from_tuples(tuples, names=['Merkmalcluster', 'MerkmalNr'])
df = df_merkmale.set_index(index)
df

In [ ]:
for merkmal in df['Merkmalcluster'].unique():
    print('generate number for merkmal:',merkmal)

In [ ]:
df.query("Merkmalcluster == 8")['MerkmalNr'].astype(str).values

In [ ]:
# get set of Merkmale
setMerkmale = df.query("Merkmalcluster == 8")['MerkmalNr'].astype(str).values
# ignoe 'no ...' part
unvalidClusters = [element for element in setMerkmale if '99999' in element]
setMerkmale = setMerkmale[setMerkmale != unvalidClusters]

In [ ]:
[element for element in setMerkmale if 99999 in element]

In [ ]:
setMerkmale

In [ ]:
df.query("Merkmalcluster == 8")

In [ ]:
# select random element
randomElementInList(setMerkmale)

In [ ]:
# build bike
bike = []
bikePrice = 0
for cluster in df['Merkmalcluster'].unique():
    # get set of Merkmale
    setMerkmale = df.query("Merkmalcluster == " + str(cluster))['MerkmalNr'].values    
    # ignoe 'no ...' part (marked with MerkmalNr : 'xxx99999')
    unvalidClusters = [element for element in setMerkmale.astype(str) if '99999' in element]
    setMerkmale = setMerkmale[setMerkmale != unvalidClusters]
    # select random bike part
    bikePart = randomElementInList(setMerkmale)
    # get price of this part
    pricePart = df.swaplevel().loc[int(bikePart)]['Preis'].astype(float).iloc[0]
    # sum it up to get the total bike price
    bikePrice += pricePart
    bike.append(bikePart)
bike, bikePrice

In [ ]:
# TODO: generate insert

In [ ]:
df.swaplevel().loc[100001]['Preis']

In [ ]:
df.swaplevel().loc[100003]['Preis'].astype(float).iloc[0]

In [ ]:
[element for element in setMerkmale.astype(str) if '99999' in element]

In [ ]:
setMerkmale[setMerkmale.astype(str) != unvalidClusters]

In [ ]:
setMerkmale[setMerkmale != unvalidClusters]